In [ ]:
import argparse
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import ollama
from langchain_community.vectorstores import Chroma
from langchain_community import embeddings
from langchain_community.embeddings.ollama import OllamaEmbeddings
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
from langchain_text_splitters import CharacterTextSplitter





ModuleNotFoundError: No module named 'langchain_community'

In [ ]:
with open("SOMAES_SPLIT.txt") as f:
    text_file_data= f.read()

In [ ]:
# Define the file path and the split keyword
file_path = "SOMAES_SPLIT.txt"
split_keyword = "[SPLIT]"

# Read the content of the file
with open(file_path, 'r', encoding='utf-8') as file:
    content = file.read()

# Split the content based on the keyword
segments = content.split(split_keyword)

#undo

In [ ]:
print(segments)

In [ ]:

model = SentenceTransformer('all-MiniLM-L6-v2')


NameError: name 'SentenceTransformer' is not defined

In [ ]:
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings

# Initialize the Sentence-BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Ensure page_of_pdf_clear is already a list of strings
# Example:
# page_of_pdf_clear = ["First page text", "Second page text", ...]
doc_list_to_embed = segments
# Create embeddings for the documents
embeddings = model.encode(doc_list_to_embed, convert_to_numpy=True)  # Directly to numpy array

# Convert embeddings from numpy arrays to lists
embeddings = embeddings.tolist()

# Initialize Chroma client and collection
chroma_client = chromadb.Client(Settings(persist_directory="/docs/chroma_2"))
collection = chroma_client.get_or_create_collection("documents")

# Add documents and their embeddings to the collection
for i, (doc, emb) in enumerate(zip(doc_list_to_embed, embeddings)):
    doc_id = f"doc_{i}"
    collection.add(ids=[doc_id], documents=[doc], embeddings=[emb])



ModuleNotFoundError: No module named 'sentence_transformers'

In [ ]:
# Initialize the Sentence-BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Create embeddings for the documents
embeddings = model.encode(doc_list_to_embed, convert_to_numpy=True)

NameError: name 'SentenceTransformer' is not defined

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Function to perform similarity search
def similarity_search(query, embeddings, documents, top_k=5):
    # Generate embedding for the query
    query_embedding = model.encode([query], convert_to_numpy=True)

    # Calculate cosine similarities between the query embedding and all document embeddings
    similarities = cosine_similarity(query_embedding, embeddings).flatten()

    # Get the top_k indices of the most similar documents
    top_k_indices = similarities.argsort()[-top_k:][::-1]

    # Retrieve the most similar documents and their similarity scores
    results = {
        "documents": [documents[idx] for idx in top_k_indices],
        "similarities": [similarities[idx] for idx in top_k_indices]
    }

    return results



ModuleNotFoundError: No module named 'sentence_transformers'

In [ ]:
#add query here
query = "who is the president of somaes"
top_k = 5  # Number of similar documents to retrieve
new_context = similarity_search(query, embeddings, doc_list_to_embed, top_k)["documents"]


NameError: name 'similarity_search' is not defined

In [ ]:
for i in range(5):
    print(new_context[i])
    print()
    print()

In [ ]:
modela = ChatOllama(model = "llama3")
prompt_template = """Answer the question directly without mentioning or referring to the provided context. If the answer is not found in the context, respond by indicating that the information is not available and ask the user to clarify the question if possible
Context: {context}
Question: {question}"""
prompt_template_final = ChatPromptTemplate.from_template(prompt_template)
prompt = prompt_template_final.format(context=new_context, question=query)
answer = modela.invoke(prompt)
print(answer.content)

NameError: name 'ChatOllama' is not defined